In [1]:
import telegram
import requests as req
import json
import xml.etree.ElementTree as et

In [2]:
# 텔레그램 인증키 등록
token = '2131981568:AAEeNzscHJkDMPHKd59X9tCvd-0UNckUx6c'

In [3]:
bot = telegram.Bot(token=token)

In [4]:
from telegram import Update
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters
from telegram.ext import Updater, CallbackContext

### 코로나 시도별 발생현황 살펴보기
* 시도를 입력하면 "확진자수, 격리중 환자수, 해외유입 수, 전일대비 증감 수, 격리 해제 수"등을 출력함 

In [5]:
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey=%2BpJJikYO7sLu5IRIGSVwrRhI2csJLgnXqbiIJsxf6n2g8HQ7IZwYWfBdGUhI5pclA093YhJpUyLroFhTYVfL8g%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200410&endCreateDt=20211123'

In [6]:
res = req.get(url)
res.text[:500]

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><createDt>2021-11-23 08:57:17.931</createDt><deathCnt>16</deathCnt><defCnt>6407</defCnt><gubun>검역</gubun><gubunCn>隔離區</gubunCn><gubunEn>Lazaretto</gubunEn><incDec>1</incDec><isolClearCnt>6253</isolClearCnt><isolIngCnt>138</isolIngCnt><localOccCnt>0</localOccCnt><overFlowCnt>1</overFlowCnt><qurRate>-</qurRate><seq>13461</seq><stdDay>'

In [7]:
# 받은 결과를 파일로 저장
with open('/home/hadoop/telegram/covid.xml','w',encoding='UTF-8') as f:
    f.writelines(res.text)

In [8]:
# xml 파일 읽어 XML 분석하고 DOM 트리를 구축함
tree = et.parse('/home/hadoop/telegram/covid.xml')
root = tree.getroot()

In [9]:
# # gu = '서울'
# for data in root.findall('body/items/item'):
#     # if (data.find('gubun').text == gu):
#         날짜 = data.find('createDt').text
#         지역 = data.find('gubun').text
#         확진자수 = data.find('defCnt').text
#         격리중환자수 = data.find('isolIngCnt').text
#         해외유입수 = data.find('overFlowCnt').text
#         전일대비 = data.find('incDec').text
#         격리해제 = data.find('isolClearCnt').text
#         print(날짜, 지역, 확진자수, 격리중환자수, 해외유입수, 전일대비,격리해제)

In [10]:
sd = '서울'
cnt = 0 # 지역과 종합 코로나 정보를 출력하기 위한 
for data in root.findall('body/items/item'):
    if (data.find('gubun').text == sd) or \
    (data.find('gubun').text == '합계'):
        날짜 = data.find('createDt').text
        확진자수 = data.find('defCnt').text
        격리중환자수 = data.find('isolIngCnt').text
        해외유입수 = data.find('overFlowCnt').text
        전일대비 = data.find('incDec').text
        격리해제 = data.find('isolClearCnt').text
        print(날짜, 확진자수, 격리중환자수, 해외유입수, 전일대비,격리해제)
        cnt = cnt+1
    if cnt == 2: break

2021-11-23 08:57:17.927 142913 17992 5 1165 123948
2021-11-23 08:57:17.927 420950 39061 14 2699 378561


### 텔레그램과 연동햐서 코로나 정보 출력하기

In [11]:
from telegram import Update
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters
from telegram.ext import Updater, CallbackContext

In [13]:
# 특정 대화상대에게 메시지 보내기 테스트
# sendMessage(대상, 메시지)
bot.sendMessage(chat_id='2110692703', text='hello, World!')

In [14]:
# 텔레그램으로 미세먼지 정보 서비스 하기
def reload_covid(): # data.go.kr 에서 대기정보 받아오기
    res = req.get(url)
    with open('/home/hadoop/telegram/covid19.xml', 'w', encoding='UTF-8') as f:
        f.writelines(res.text)
        

def fetch_covid(sido): # 특정지역의 미세먼지 정보/이모지 받아오기
    tree = et.parse('/home/hadoop/telegram/covid.xml')
    root = tree.getroot()
    
    covidinfo = ''
    cnt = 0  
    for data in root.findall('body/items/item'):
        if (data.find('gubun').text == sd) or (data.find('gubun').text == '합계'):
            지역 = data.find('gubun').text
            날짜 = data.find('createDt').text[:10]
            확진자수 = data.find('defCnt').text
            격리중환자수 = data.find('isolIngCnt').text
            해외유입수 = data.find('overFlowCnt').text
            전일대비 = data.find('incDec').text
            격리해제 = data.find('isolClearCnt').text
            covidinfo +=    f'지역 :{지역}\n' \
                            f'확진자수 :{확진자수}\n' \
                            f'전일대비 증감수 :{전일대비}\n' 
            cnt = cnt+1
        if cnt == 2: break
    return covidinfo

In [15]:
# 대기정보 출력
def covid_message(update, context):
    covidinfo = fetch_covid(update.message.text)
    context.bot.send_message(update.effective_chat.id, covidinfo)

In [16]:
# 대기정보 갱신
def reload_message(update, context):
    reload_covid()
    context.bot.send_message(update.effective_chat.id, '대기정보 갱신됨!!')

In [17]:
# 텔레그램 챗봇 서버 초기화
updater = Updater(token)

# 핸들러 생성
covid_handler = MessageHandler(Filters.text, covid_message)
reload_handler = CommandHandler('reload', reload_message)

# 핸들러 등록
updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(covid_handler)

In [ ]:
# 텔레그램 챗봇 서버 시작
print('텔레그램 대기정보 RPA 서버 실행중...')
updater.start_polling(timeout=3, clean=True)
updater.idle()
# 텔레그램 구동 -> /reload -> 구이름

텔레그램 대기정보 RPA 서버 실행중...


/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
